# Run Dreambooth fine-tuned models for Stable Diffusion using d🧨ffusers 

This notebook allows you to run Stable Diffusion concepts trained via Dreambooth using 🤗 Hugging Face [🧨 Diffusers library](https://github.com/huggingface/diffusers). 

Train your own using [here](#) and navigate the [public library concepts](#) to pick yours. You may also want to use the [Spaces](#) to browse the library


![Dreambooth Example](https://dreambooth.github.io/DreamBooth_files/teaser_static.jpg)
_By using just 3-5 images you can teach new concepts to Stable Diffusion and personalize the model on your own images_ 

Differently from Textual Inversion, this approach trains the whole model, which can yield better results to the cost of bigger models.

In [ ]:
#@markdown Check type of GPU and VRAM available.
!nvidia-smi --query-gpu=name,memory.total,memory.free --format=csv,noheader

Tesla T4, 15109 MiB, 15109 MiB


https://github.com/feynlee/Stable_Diffusion/dreambooth

This notebook is adapted from [ShivamShrirao's notebook](https://github.com/ShivamShrirao/diffusers/tree/main/examples/dreambooth).

# Training

## Install Requirements

In [ ]:
!wget -q https://github.com/ShivamShrirao/diffusers/raw/main/examples/dreambooth/train_dreambooth.py
%pip install -qq git+https://github.com/ShivamShrirao/diffusers
%pip install -q -U --pre triton
%pip install -q accelerate==0.12.0 transformers ftfy bitsandbytes gradio

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 163 kB 5.1 MB/s 
     |████████████████████████████████| 18.7 MB 1.3 MB/s 
     |████████████████████████████████| 143 kB 5.2 MB/s 
     |████████████████████████████████| 4.9 MB 50.9 MB/s 
     |████████████████████████████████| 53 kB 1.5 MB/s 
     |████████████████████████████████| 55.9 MB 147 kB/s 
     |████████████████████████████████| 5.3 MB 49.7 MB/s 
     |████████████████████████████████| 6.6 MB 40.7 MB/s 
     |████████████████████████████████| 84 kB 3.1 MB/s 
     |████████████████████████████████| 84 kB 3.4 MB/s 
     |████████████████████████████████| 270 kB 67.2 MB/s 
     |████████████████████████████████| 55 kB 3.0 MB/s 
     |████████████████████████████████| 57 kB 4.1 MB/s 
     |████████████████████████████████| 212 kB 70.4 MB/s 
     |████████████████████████████████| 2.3 MB 51.2 MB/s 
     |██████████

In [ ]:
#@title Login to HuggingFace 🤗

#@markdown You need to accept the model license before downloading or using the Stable Diffusion weights. Please, visit the [model card](https://huggingface.co/CompVis/stable-diffusion-v1-4), read the license and tick the checkbox if you agree. You have to be a registered user in 🤗 Hugging Face Hub, and you'll also need to use an access token for the code to work.
from huggingface_hub import notebook_login
!git config --global credential.helper store
notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token


### Install xformers from precompiled wheel.

In [ ]:
%pip install -q https://github.com/metrolobo/xformers_wheels/releases/download/1d31a3ac_various_6/xformers-0.0.14.dev0-cp37-cp37m-linux_x86_64.whl
# These were compiled on Tesla T4, should also work on P100, thanks to https://github.com/metrolobo

# If precompiled wheels don't work, install it with the following command. It will take around 40 minutes to compile.
# %pip install git+https://github.com/facebookresearch/xformers@1d31a3a#egg=xformers

     |████████████████████████████████| 102.9 MB 88 kB/s 


## Settings and run

In [ ]:
#@markdown Name/Path of the initial model.
MODEL_NAME = "sd-dreambooth-library/disco-diffusion-style" #@param {type:"string"}
# default: CompVis/stable-diffusion-v1-4

#@markdown Path for images of the concept for training.
INSTANCE_DIR = "/content/drive/MyDrive/AI/my_photos" #@param {type:"string"}

# #@markdown A general name for class like dog for dog images.
# CLASS_NAME = "person" #@param {type:"string"}
CLASS_DIR = f"/content/class_images"

#@markdown If model weights should be saved directly in google drive (takes around 4-5 GB).
save_to_gdrive = True #@param {type:"boolean"}
if save_to_gdrive:
    from google.colab import drive
    # make sure google drive is unmounted before remount
    drive.mount('/content/drive')

#@markdown Enter the directory name to save model at.
OUTPUT_DIR = "stable_diffusion_weights/sks" #@param {type:"string"}
if save_to_gdrive:
    OUTPUT_DIR = "/content/drive/MyDrive/" + OUTPUT_DIR
else:
    OUTPUT_DIR = "/content/" + OUTPUT_DIR

print(f"[*] Weights will be saved at {OUTPUT_DIR}")

!mkdir -p $INSTANCE_DIR
!mkdir -p $OUTPUT_DIR

#@markdown sks is a rare identifier, feel free to replace it.

Mounted at /content/drive
[*] Weights will be saved at /content/drive/MyDrive/stable_diffusion_weights/liziyue


Upload your (5-6) images to the INSTANCE_DIR you specified above.

## Start Training

Use the table below to choose the best flags based on your memory and speed requirements. Tested on Tesla T4 GPU.


| `fp16` | `train_batch_size` | `gradient_accumulation_steps` | `gradient_checkpointing` | `use_8bit_adam` | GB VRAM usage | Speed (it/s) |
| ---- | ------------------ | ----------------------------- | ----------------------- | --------------- | ---------- | ------------ |
| fp16 | 1                  | 1                             | TRUE                    | TRUE            | 9.92       | 0.93         |
| no   | 1                  | 1                             | TRUE                    | TRUE            | 10.08      | 0.42         |
| fp16 | 2                  | 1                             | TRUE                    | TRUE            | 10.4       | 0.66         |
| fp16 | 1                  | 1                             | FALSE                   | TRUE            | 11.17      | 1.14         |
| no   | 1                  | 1                             | FALSE                   | TRUE            | 11.17      | 0.49         |
| fp16 | 1                  | 2                             | TRUE                    | TRUE            | 11.56      | 1            |
| fp16 | 2                  | 1                             | FALSE                   | TRUE            | 13.67      | 0.82         |
| fp16 | 1                  | 2                             | FALSE                   | TRUE            | 13.7       | 0.83          |
| fp16 | 1                  | 1                             | TRUE                    | FALSE           | 15.79      | 0.77         |


In [ ]:
#@title Setting up all training args
from argparse import Namespace

args = Namespace(
    pretrained_model_name_or_path=MODEL_NAME,
    center_crop=True,
    instance_data_dir=INSTANCE_DIR,
    #@markdown `instance_prompt` is a prompt that should contain a good description of what your object or style is
    instance_prompt="portrait photo of sks guy"  #@param {type:"string"}
    ,
    #@markdown `class_prompt` is a prompt that should contain a good description of what the class your object belongs to
    class_prompt="portrait photo of a guy"  #@param {type:"string"}
    ,
    resolution=512 #@param {type:"number"}
    ,
    learning_rate=3e-06  #@param {type:"number"}
    ,
    max_train_steps=2000  #@param {type:"number"}
    ,
    train_batch_size=1,
    gradient_accumulation_steps=1,
    max_grad_norm=1.0,
    mixed_precision="fp16", 
    prior_loss_weight=1.0  #@param {type:"number"}
    ,
    sample_batch_size=4, # default: 2
    class_data_dir=CLASS_DIR, 
    num_class_images=40  #@param {type:"number"}
    , 
    output_dir=OUTPUT_DIR,
    lr_scheduler="constant",
    lr_warmup_steps=0,
    seed=1337 #@param {type:"number"}
    , 
    #@markdown If you wish your model to be avaliable for everyone, push it to the public library. If you prefer to use your model privately, add your own profile. Once you save it you can use your concept by loading the model on any from_pretrained function
    push_to_hub=True #@param {type:"boolean"}
    , 
    hub_model_id="sd-sks" #@param {type:"string"}
    ,
    #@markdown `hub_token` leave blank if you logged in with a token with `write access` in the [Initial Setup](#scrollTo=KbzZ9xe6dWwf). If not, [go to your tokens settings and create a write access token](https://huggingface.co/settings/tokens)
    hub_token="" #@param {type:"string"}
)

In [ ]:
#@markdown Add `--gradient_checkpointing` flag for around 9.92 GB VRAM usage.

#@markdown remove `--use_8bit_adam` flag for full precision. Requires 15.79 GB with `--gradient_checkpointing` else 17.8 GB.
if args.push_to_hub:
    !accelerate launch train_dreambooth.py \
    --pretrained_model_name_or_path=$MODEL_NAME \
    --instance_data_dir=$INSTANCE_DIR \
    --class_data_dir=$CLASS_DIR \
    --output_dir=$OUTPUT_DIR \
    --with_prior_preservation --prior_loss_weight=$args.prior_loss_weight \
    --instance_prompt="$args.instance_prompt" \
    --class_prompt="$args.class_prompt" \
    --seed=$args.seed \
    --resolution=$args.resolution \
    --center_crop \
    --train_batch_size=$args.train_batch_size \
    --mixed_precision=$args.mixed_precision \
    --use_8bit_adam \
    --gradient_accumulation_steps=1 \
    --learning_rate=$args.learning_rate \
    --lr_scheduler=$args.lr_scheduler \
    --lr_warmup_steps=$args.lr_warmup_steps \
    --num_class_images=$args.num_class_images \
    --sample_batch_size=$args.sample_batch_size \
    --max_train_steps=$args.max_train_steps \
    --push_to_hub \
    --hub_model_id=$args.hub_model_id \
    --hub_token=$args.hub_token 
else:
    !accelerate launch train_dreambooth.py \
    --pretrained_model_name_or_path=$MODEL_NAME \
    --instance_data_dir=$INSTANCE_DIR \
    --class_data_dir=$CLASS_DIR \
    --output_dir=$OUTPUT_DIR \
    --with_prior_preservation --prior_loss_weight=$args.prior_loss_weight \
    --instance_prompt="$args.instance_prompt" \
    --class_prompt="$args.class_prompt" \
    --seed=$args.seed \
    --resolution=$args.resolution \
    --center_crop \
    --train_batch_size=$args.train_batch_size \
    --mixed_precision=$args.mixed_precision \
    --use_8bit_adam \
    --gradient_accumulation_steps=1 \
    --learning_rate=$args.learning_rate \
    --lr_scheduler=$args.lr_scheduler \
    --lr_warmup_steps=$args.lr_warmup_steps \
    --num_class_images=$args.num_class_images \
    --sample_batch_size=$args.sample_batch_size \
    --max_train_steps=$args.max_train_steps 

The following values were not passed to `accelerate launch` and had defaults used instead:
	`--num_processes` was set to a value of `1`
	`--num_machines` was set to a value of `1`
	`--mixed_precision` was set to a value of `'no'`
	`--num_cpu_threads_per_process` was set to `1` to improve out-of-box performance
To avoid this warning pass in values for each of the problematic parameters or run `accelerate config`.
Traceback (most recent call last):
  File "train_dreambooth.py", line 637, in <module>
    main()
  File "train_dreambooth.py", line 348, in main
    logging_dir=logging_dir,
  File "/usr/local/lib/python3.7/dist-packages/accelerate/accelerator.py", line 286, in __init__
    raise ValueError(err.format(mode="fp16", requirement="a GPU"))
ValueError: fp16 mixed precision requires a GPU
Traceback (most recent call last):
  File "/usr/local/bin/accelerate", line 8, in <module>
    sys.exit(main())
  File "/usr/local/lib/python3.7/dist-packages/accelerate/commands/accelerate_cli.py"

## Convert weights to ckpt to use in web UIs like AUTOMATIC1111.

In [ ]:
#@markdown Download script
!wget -q https://gist.github.com/jachiam/8a5c0b607e38fcc585168b90c686eb05/raw/2af0a9c0237ed98b863a75e1db21d7ed5541094f/convert_diffusers_to_sd.py

In [ ]:
#@markdown Run conversion.
model_name = "model-sks" #@param {type: "string"}
ckpt_path = OUTPUT_DIR + f"/{model_name}.ckpt"

half_arg = ""
#@markdown  Whether to convert to fp16, takes half the space (2GB), might loose some quality.
fp16 = False #@param {type: "boolean"}
if fp16:
    half_arg = "--half"
!python convert_diffusers_to_sd.py --model_path $OUTPUT_DIR  --checkpoint_path $ckpt_path $half_arg
print(f"[*] Converted ckpt saved at {ckpt_path}")

Reshaping encoder.mid.attn_1.q.weight for SD format
Reshaping encoder.mid.attn_1.k.weight for SD format
Reshaping encoder.mid.attn_1.v.weight for SD format
Reshaping encoder.mid.attn_1.proj_out.weight for SD format
Reshaping decoder.mid.attn_1.q.weight for SD format
Reshaping decoder.mid.attn_1.k.weight for SD format
Reshaping decoder.mid.attn_1.v.weight for SD format
Reshaping decoder.mid.attn_1.proj_out.weight for SD format
[*] Converted ckpt saved at /content/drive/MyDrive/stable_diffusion_weights/sks/model.ckpt


# Save to HuggingFace Hub

In [ ]:
#@title Save your newly created concept? you may save it privately to your personal profile or collaborate to the [library of concepts](https://huggingface.co/sd-dreambooth-library)?
#@markdown If you wish your model to be avaliable for everyone, add it to the public library. If you prefer to use your model privately, add your own profile.
import os
import torch
from diffusers import StableDiffusionPipeline

save_concept = True #@param {type:"boolean"}
#@markdown Once you save it you can use your concept by loading the model on any `from_pretrained` function
name_of_your_concept = "sks" #@param {type:"string"}
where_to_save_concept = "privately_to_my_profile" #@param ["public_library", "privately_to_my_profile"]

#@markdown `TOKEN`: leave blank if you logged in with a token with `write access` in the Initial Setup. If not, [go to your tokens settings and create a write access token](https://huggingface.co/settings/tokens)
TOKEN = "" #@param {type:"string"}
if(save_concept):
  from slugify import slugify
  from huggingface_hub import HfApi, HfFolder, CommitOperationAdd
  from huggingface_hub import create_repo
  from IPython.display import display_markdown
  api = HfApi()
  your_username = api.whoami(token=TOKEN)["name"]
  pipe = StableDiffusionPipeline.from_pretrained(
    OUTPUT_DIR,
    torch_dtype=torch.float16,
  ).to("cuda")
  os.makedirs("fp16_model",exist_ok=True)
  pipe.save_pretrained("fp16_model")

  if(where_to_save_concept == "public_library"):
    repo_id = f"sd-dreambooth-library/{slugify(name_of_your_concept)}"
    #Join the Concepts Library organization if you aren't part of it already
    !curl -X POST -H 'Authorization: Bearer '$hf_token -H 'Content-Type: application/json' https://huggingface.co/organizations/sd-dreambooth-library/share/SSeOwppVCscfTEzFGQaqpfcjukVeNrKNHX
  else:
    repo_id = f"{your_username}/{slugify(name_of_your_concept)}"
  output_dir = OUTPUT_DIR
  if(not TOKEN):
    with open(HfFolder.path_token, 'r') as fin: hf_token = fin.read();
  else:
    hf_token = TOKEN
  
  images_upload = os.listdir(args.instance_data_dir)
  image_string = ""
  #repo_id = f"sd-dreambooth-library/{slugify(name_of_your_concept)}"
  for i, image in enumerate(images_upload):
      image_string = f'''{image_string}![image {i}](https://huggingface.co/{repo_id}/resolve/main/concept_images/{image})
'''
  readme_text = f'''---
license: mit
---
### {name_of_your_concept} on Stable Diffusion via Dreambooth
#### model by {api.whoami(token=TOKEN)["name"]}
This your the Stable Diffusion model fine-tuned the {name_of_your_concept} concept taught to Stable Diffusion with Dreambooth.
It can be used by modifying the `instance_prompt`: **{args.instance_prompt}**

You can also train your own concepts and upload them to the library by using [this notebook](https://colab.research.google.com/github/huggingface/notebooks/blob/main/diffusers/sd_dreambooth_training.ipynb).
And you can run your new concept via `diffusers`: [Colab Notebook for Inference](https://colab.research.google.com/github/huggingface/notebooks/blob/main/diffusers/sd_dreambooth_inference.ipynb), [Spaces with the Public Concepts loaded](https://huggingface.co/spaces/sd-dreambooth-library/stable-diffusion-dreambooth-concepts)

Here are the images used for training this concept:
{image_string}
'''
  #Save the readme to a file
  readme_file = open("README.md", "w")
  readme_file.write(readme_text)
  readme_file.close()
  #Save the token identifier to a file
  text_file = open("token_identifier.txt", "w")
  text_file.write(args.instance_prompt)
  text_file.close()
  operations = [
    CommitOperationAdd(path_in_repo="token_identifier.txt", path_or_fileobj="token_identifier.txt"),
    CommitOperationAdd(path_in_repo="README.md", path_or_fileobj="README.md"),
  ]
  try:
    create_repo(repo_id,private=True, token=hf_token)
  except:
    pass
  
  api.create_commit(
    repo_id=repo_id,
    operations=operations,
    commit_message=f"Upload the concept {name_of_your_concept} embeds and token",
    token=hf_token
  )
  api.upload_folder(
    folder_path="fp16_model",
    path_in_repo="",
    repo_id=repo_id,
    token=hf_token
  )
  api.upload_folder(
    folder_path=args.instance_data_dir,
    path_in_repo="concept_images",
    repo_id=repo_id,
    token=hf_token
  )
display_markdown(f'''## Your concept was saved successfully. [Click here to access it](https://huggingface.co/{repo_id})
''', raw=True)

## Your concept was saved successfully. [Click here to access it](https://huggingface.co/Feynlee/liziyue-2000)


# Inference

## Setup

In [ ]:
import torch
from torch import autocast
from diffusers import StableDiffusionPipeline
from IPython.display import display

#@markdown You can use the HuggingFace repo_id where you saved the model. If you keep this empty, it points to the path of model in gdrive.
model_path = ""    #@param {type:"string"}
if model_path == "":
    model_path = OUTPUT_DIR
#@markdown TOKEN should have read access HuggingFace
TOKEN = "" #@param {type:"string"}

pipe = StableDiffusionPipeline.from_pretrained(model_path, torch_dtype=torch.float16, use_auth_token=TOKEN).to("cuda")
g_cuda = None

In [ ]:
#@markdown Can set random seed here for reproducibility.
g_cuda = torch.Generator(device='cuda')
seed = 52362 #@param {type:"number"}
g_cuda.manual_seed(seed)

In [ ]:
#@title Run the Stable Diffusion pipeline on Colab
#@markdown Don't forget to use the placeholder token in your prompt
import math
from PIL import Image
from torch import autocast
prompt = "sks digital painting, portrait, trending on artstation" #@param {type:"string"}

num_samples = 4 #@param {type:"number"}
num_batches = 1 #@param {type:"number"}
num_columns = 2 #@param {type:"number"}
guidance_scale = 4 #@param {type:"number"}
num_inference_steps = 50 #@param {type:"number"}
height = 512 #@param {type:"number"}
width = 512 #@param {type:"number"}


def image_grid(imgs, cols):
    total = len(imgs)
    rows = math.ceil(total / cols)

    w, h = imgs[0].size
    grid = Image.new('RGB', size=(cols*w, rows*h))
    grid_w, grid_h = grid.size
    
    for i, img in enumerate(imgs):
        grid.paste(img, box=(i%cols*w, i//cols*h))
    return grid

all_images = [] 
for _ in range(num_batches):
    with autocast("cuda"):
        images = pipe([prompt] * num_samples, height=height, width=width, num_inference_steps=num_inference_steps, guidance_scale=guidance_scale).images
        all_images.extend(images)


grid = image_grid(all_images, num_columns)
grid